In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import heartpy as hp
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal, interpolate
import copy
import os 
import pandas as pd
os.chdir('/home/kkotzen/research/PPG_sleepstaging_orion3/')
from pathlib import Path
from src.parsing.SpanishParser import SpanishParser
from scipy.signal import butter, sosfiltfilt, resample


In [26]:
from src.parsing.utils.comp_ppg_signal_quality import comp_ppg_signal_quality_using_PPG_BSQI

dl = SpanishParser()
patient = dl.database_all_patient_IDs[222]
ppg = dl.load_signal(patient,'Pleth')
strong_ppg_points = dl.load_annotation(patient, 'Pleth', 'Neurokit', 'Peaks')
weak_ppg_points = dl.load_annotation(patient, 'Pleth', 'Billauer', 'Peaks')
quality = comp_ppg_signal_quality_using_PPG_BSQI(strong_ppg_points, weak_ppg_points, len(ppg), fs=512).quality.values

plt.close('all')
t_ppg = np.arange(0, len(ppg)/512, 1/512)
t_quality = np.arange(0, len(quality)*30, 30)
ax = plt.subplot(2,1,1)
plt.plot(t_ppg, ppg)
plt.plot(strong_ppg_points/512, ppg[strong_ppg_points], 'x')
plt.plot(weak_ppg_points/512, ppg[weak_ppg_points], 'o')

plt.subplot(2,1,2, sharex=ax)
plt.plot(t_quality, quality)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
350-29

321

7.417777777777777

In [ ]:
# fs = 512
# dl = SpanishParser()
# patients = dl.database_all_patient_IDs
# ecg = dl.load_signal(patients[300], 'ECG')[0:512*100*60]
# working_data, metrics = hp.process(ecg, sample_rate = 512)
# plt.close('all')
# hp.plotter(working_data, metrics)

# sos = butter(8, [0.5 / (fs / 2), 25 / (fs / 2)], 'bandpass', output='sos')
# ecg_filt = sosfiltfilt(sos, ecg, axis=0)  # , padlen=3 * (max(len(b), len(a)) - 1))
# ecg_resample = resample(ecg_filt, int(len(ecg_filt)/8))


In [ ]:
def comp_google_ihr(beats, fs, resample_fs):

    beats = beats / fs

    ibi = np.diff(beats)
    beats = beats[:-1] + (beats[1:] - beats[:-1]) / 2

    upper = np.percentile(ibi, 99)
    lower = np.percentile(ibi, 1)

    ibi_filt_idx = np.where((ibi < upper) & (ibi > lower))
    ibi = ibi[ibi_filt_idx]
    beats = beats[ibi_filt_idx]
    ihr = 1 / ibi

    x_2hz = np.arange(0, beats[-1], 1 / resample_fs)
    ihr_2hz = np.interp(x_2hz, beats, ihr)

    ihr_2hz = (ihr_2hz - np.mean(ihr_2hz)) / np.std(ihr_2hz)

    return ihr_2hz

dl = SpanishParser()
patients = dl.database_all_patient_IDs

ppg = dl.load_signal(patient, 'Pleth')
beats = dl.load_annotation(patient, 'Pleth', 'Bishop', 'Peaks')
sleep = dl.load_sleep(patient)

ihr = comp_google_ihr(beats, 512, 2)

plt.close('all')
# plt.plot(ppg)
# plt.plot(beats, ppg[beats], 'x')
plt.close('all')
t_ihr = np.arange(0, len(ihr)/2, 1/2)
t_sleep = np.arange(0, len(sleep)*30, 30)
ax1 = plt.subplot(2,1,1)
plt.plot(t_ihr, ihr)
ax2 = plt.subplot(2,1,2, sharex=ax1)
plt.plot(t_sleep, sleep)


In [ ]:
dl = SpanishParser()
patients = dl.database_all_patient_IDs


patient = patients[4]
fs = 512
print(dl.path_signal(patient, 'Pleth'))
ppg = dl.load_signal(patient, 'Pleth')
ecg = dl.load_signal(patient, 'ECG')

peaks = dl.load_annotation(patient, 'Pleth', 'Bishop', 'Peaks').astype(int)
sleep = dl.load_sleep(patient)

sos = butter(8, [0.5 / (fs / 2), 8 / (fs / 2)], 'bandpass', output='sos')
ppg_filt = sosfiltfilt(sos, ppg, axis=0)  # , padlen=3 * (max(len(b), len(a)) - 1))
plt.close("all")
plt.plot(ppg_filt)
plt.plot(peaks, ppg_filt[peaks], 'x')

In [9]:
from src.parsing.utils.extract_numpy_from_edf import extract_numpy_from_edf

# edf = dl.path_edf(patients[12])
# ecg1 = extract_numpy_from_edf(edf, 'ECG1')
# ecg2 = extract_numpy_from_edf(edf, 'ECG2')

# sos = butter(8, [0.5 / (fs / 2), 16 / (fs / 2)], 'bandpass', output='sos')
# ecg1_filt = sosfiltfilt(sos, ecg1, axis=0)
# ecg2_filt = sosfiltfilt(sos, ecg2, axis=0)

# plt.close('all')
# ax1 = plt.subplot(2,1,1)
# plt.plot(ecg1, alpha=0.5)
# plt.plot(ecg1_filt)
# ax2 = plt.subplot(2,1,2, sharex = ax1)
# plt.plot(ecg2)
# plt.plot(ecg2_filt, alpha=0.5)
# ax3 = plt.subplot(3,1,3, sharex = ax1)
# ecg = ecg2-ecg1
# plt.plot(ecg)

# plt.close('all')
# plt.plot(ecg)


from multiprocessing import Pool
from tqdm import tqdm

dl = SpanishParser()
tasks = dl.database_all_patient_IDs
# pool = Pool(processes=16)
pbar = tqdm(total=len(tasks))

def extract_ECG(patient):
    try:
        dl = SpanishParser()
        edf = dl.path_edf(patient)
        ecg1 = extract_numpy_from_edf(edf, 'ECG1')
        ecg2 = extract_numpy_from_edf(edf, 'ECG2')
        np.save(dl.path_signal(patient, 'ECG1'), ecg1)
        np.save(dl.path_signal(patient, 'ECG2'), ecg1)
#         pbar.update(1)
    except: 
        print(f"failed: {patient}")

for task in tqdm(tasks):
    extract_ECG(task)
print("Done")
# pool.imap_unordered(extract_ECG, tasks)
# pool.close()
# pool.join()
# pbar.close()








  0%|                                                                    | 0/321 [00:00<?, ?it/s]





  0%|                                                                    | 0/321 [00:00<?, ?it/s]





  0%|                                                                    | 0/321 [00:44<?, ?it/s]






  1%|▎                                                           | 2/321 [00:06<17:43,  3.33s/it]





  1%|▌                                                           | 3/321 [00:09<15:20,  2.89s/it]





  1%|▋                                                           | 4/321 [00:09<09:28,  1.79s/it]

failed: 861








  2%|▉                                                           | 5/321 [00:12<11:23,  2.16s/it]





  2%|█                                                           | 6/321 [00:35<48:26,  9.23s/it]





  2%|█▎                                                        | 7/321 [00:53<1:04:26, 12.31s/it]

failed: 871








  2%|█▍                                                        | 8/321 [01:16<1:21:36, 15.64s/it]





  3%|█▋                                                        | 9/321 [01:39<1:32:38, 17.82s/it]





  3%|█▊                                                       | 10/321 [02:02<1:41:15, 19.54s/it]





  3%|█▉                                                       | 11/321 [02:23<1:43:44, 20.08s/it]





  4%|██▏                                                      | 12/321 [02:44<1:43:53, 20.17s/it]





  4%|██▎                                                      | 13/321 [02:46<1:15:48, 14.77s/it]





  4%|██▍                                                      | 14/321 [03:05<1:21:28, 15.92s/it]





  5%|██▋                                                      | 15/321 [03:26<1:29:45, 17.60s/it]





  5%|██▊                                                      | 16/321 [03:49<1:37:04, 19.10s/it]





  5%|███                                                      | 17/

Done


In [ ]:
plt.close('all')
t_ppg = np.arange(0, len(ppg)/512, 1/512)
t_sleep = np.arange(0, len(sleep)*30, 30)
ax1 = plt.subplot(211)
ax1.plot(t_ppg, ppg_filt)
ax2 = plt.subplot(212, sharex=ax1)
ax2.plot(t_sleep, sleep)
# plt.plot(peaks, ppg_filt[peaks], 'x')
# plt.plot(onsets, ppg_filt[onsets], 'o')

In [ ]:
t = np.arange(9)
x = np.array([0,1,1,1,2,2,2,0])
y = np.array([0,1,1,1,0,2,2,0])
green = np.where(x==y)[0]
red = np.where(x!=y)[0]


In [ ]:
def color_plot(t, x, y)
    green = np.where(x==y)[0]
    red = np.where(x!=y)[0]
    for g in green:
        plt.plot([t[g], t[g+1]], [x[g], x[g]], color='lightgreen')
    for g in red:
        plt.plot([t[g], t[g+1]], [x[g], x[g]], color='red')

    for i in green:
        if i == 0:
            pass
        elif x[i] != x[i-1]:
            plt.plot([t[i], t[i]], [x[i], x[i-1]], color='lightgreen')
    for i in red:
        if i == 0:
            pass
        elif x[i] != x[i-1]:
            plt.plot([t[i], t[i]], [x[i], x[i-1]], color='red')

In [ ]:
def duplicate(x):
    return np.array([[x_] * 2 for x_ in x]).flatten()

def hold_line(t, x):
    t = [i for i in t]
    x = [i for i in x]

    if len(t) - len(x) == 0:
        t.append(t[-1] + t[-1] - t[-2])
    if len(t) - len(x) != 1:
        raise ValueError()

    t_ = duplicate(t)[1:-1]
    x_ = duplicate(x)
    return t_, x_

In [ ]:
t_, x_ = hold_line(t,x)
_, y_ = hold_line(t,y)

In [ ]:
plt.close('all')
plt.plot(t_[green], x_[green])